In [47]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [103]:
# KOBIS 역대 영화 순위 파일 읽어오기
datas = pd.read_excel('/Users/hwkim/Documents/ds_study/data/KOBIS_datas.xlsx', engine='openpyxl', header=4)

# 영화명 + 관객수만 남기기
datas = datas[['영화명', '관객수', '배급사']][:-1]
datas.tail()

,영화명,관객수,배급사
195,82년생 김지영,3679265,롯데컬처웍스(주)롯데엔터테인먼트
196,분노의 질주: 더 익스트림,3659683,유니버설픽쳐스인터내셔널 코리아(유)
197,분노의 질주: 홉스&쇼,3657627,유니버설픽쳐스인터내셔널 코리아(유)
198,모가디슈,3619496,롯데컬처웍스(주)롯데엔터테인먼트
199,인턴,3611166,워너브러더스 코리아(주)


#### 크롤링

In [120]:
# 페이지 접근
kobis = "https://www.kobis.or.kr/kobis/business/stat/boxs/findFormerBoxOfficeList.do"
driver = webdriver.Chrome()
driver.get(kobis)

# 페이지 대기
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'td_movie')))

# 연령제한 리스트 생성
ages_list = []

# 총 200개의 영화
# tr_0 부터 tr_199까지 각 영화 하나에 대한 상세 정보가 있음.
for num in range(200):
    trs_searcher = f'//tr[@id="tr_{num}"]'
    # trs_searcher = f'//tr[@id="tr_{num}" and contains(@class, "ecllip per90")]'

    trs = driver.find_elements(By.XPATH, trs_searcher)

    # 모든 trs를 순회하면서 관람등급 가져오기
    for tr in trs:
        # 링크 클릭
        movie_link = tr.find_element(By.XPATH, './/*[@id="td_movie"]/span/a') # 상세 정보 팝업 접근
        driver.execute_script("arguments[0].scrollIntoView(true);", movie_link)
        movie_link.click() # 클릭해서 팝업 열기
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # [관람등급]에 해당하는 텍스트를 출력
        strong_tags = soup.find_all('strong')
        for strong_tag in strong_tags:
            if strong_tag.text == '[관람등급]':
                rating_text = strong_tag.next_sibling.strip()
                ages_list.append(rating_text) # 리스트에서 순서대로 추가
                print(num, rating_text) #tr_0영화의 관람등급이 15세이상관람가인경우 결과는 '0, 15세이상관람가'

        # 창 닫기 (이전 창으로 돌아가기)
        driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/a[2]/span').click()
        time.sleep(2)


# WebDriver 종료
driver.quit()

0 15세이상관람가
1 15세이상관람가
2 12세이상관람가
3 12세이상관람가
4 12세이상관람가
4 12세이상관람가
5 전체관람가
6 15세이상관람가
7 12세이상관람가
7 12세이상관람가
8 12세이상관람가
9 15세이상관람가
10 15세이상관람가
11 전체관람가
11 전체관람가
12 15세이상관람가
13 15세이상관람가
14 15세이상관람가
15 12세이상관람가
16 15세이상관람가
17 15세이상관람가
18 15세이상관람가
19 12세이상관람가
20 12세이상관람가
20 12세이상관람가
21 15세이상관람가
22 12세관람가
23 12세이상관람가
23 12세이상관람가
24 15세이상관람가
25 15세이상관람가
26 12세이상관람가
26 12세이상관람가
27 12세이상관람가
28 전체관람가
28 전체관람가
29 15세이상관람가
30 12세이상관람가
31 15세이상관람가
32 12세이상관람가
33 15세이상관람가
34 15세이상관람가
35 12세이상관람가
35 12세이상관람가
36 12세이상관람가
36 12세이상관람가
37 12세이상관람가
38 15세이상관람가
39 12세이상관람가
40 12세이상관람가
41 12세이상관람가
42 12세이상관람가
43 12세이상관람가
43 12세이상관람가
44 12세이상관람가
45 15세이상관람가
46 12세이상관람가
46 12세이상관람가
47 12세이상관람가
48 12세이상관람가
49 15세이상관람가
50 15세이상관람가
51 15세이상관람가
52 12세이상관람가
52 12세이상관람가
53 12세이상관람가
54 15세이상관람가
55 12세이상관람가
56 12세이상관람가
56 12세이상관람가
57 전체관람가
57 전체관람가
58 15세이상관람가
59 15세이상관람가
60 15세이상관람가
61 12세이상관람가
62 12세이상관람가
62 12세이상관람가
63 청소년관람불가
64 12세이상관람가
64 12세이상관람가
65 15세이상관람가
66 15세이상관람가
67 15세이상관람가
68 청소년관람불가
69 15세이상관람가
70 12

In [126]:
tables = pd.DataFrame(ages_list)
tables[:200]

,0
0,15세이상관람가
1,15세이상관람가
2,12세이상관람가
3,12세이상관람가
4,12세이상관람가
...,...
195,12세이상관람가
196,15세이상관람가
197,12세이상관람가
198,청소년관람불가


-> 총 결과가 200개여야하는데, 결과가 계속 248개로 나옴
-> 페이지 접근, 팝업창 열고 닫는 것까지는 위 코드 활용하면 될 듯   
-> 팝업 창 열었을 때, **요약정보**에서 시간, 장르, 관람제한연령 다 떠서, 여기서 한 번에 가져오도록 코드 수정해도 될 것 같음